## Make it modular

In [ ]:
import os
# Set the folder path where the files are located
folder_path = "/Users/mahtabsyed/Library/Mobile Documents/com~apple~CloudDocs/All Family Storage/1 - Career - Mahtab/UST/Hiring/Macquarie/Test/"

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    print(file_path)

In [ ]:
import os
from PyPDF2 import PdfReader
from docx import Document

# Extract selected text based on condition in function visitor_body

#  Returns an array
def remove_empty_strings(string_list):
    return [string for string in string_list if string.strip()]

parts = []

def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    # if y > 350 and y < 720: # Change this to pick Title with Big or small fonts
    if y > 1 and y < 1000:
        parts.append(text)

def extractTextPDForDOCX(file_path, filename):
    document_text = ""

    # Check if the file is a pdf document
    if filename.endswith(".pdf"):
        with open(file_path, "rb") as pdf_file:
            pdf_reader = PdfReader(pdf_file)
            num_pages = len(pdf_reader.pages)

            # Extract text from each page
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text = page.extract_text(visitor_text=visitor_body)
                # print(text)
            string_list = remove_empty_strings(parts)
       
        for string in string_list:
            document_text = document_text + string + "\n"
        
    
    # Check if the file is a Word document
    elif filename.endswith(".docx"):
        doc = Document(file_path)
        # Extract text from each paragraph
        for paragraph in doc.paragraphs:
             document_text = document_text + paragraph.text + "\n"
    
    return document_text   

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')
# print(openai.api_key )

# If using OpenAI library version 1.0.0, here is the code
# openai==1.0.0
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
# Set the folder path where the files are located
folder_path = "/Users/mahtabsyed/Library/Mobile Documents/com~apple~CloudDocs/All Family Storage/1 - Career - Mahtab/UST/Hiring/Macquarie/Test/"

jobdesc = """
Job Description
To be successful in this role you will have:
• Strong hands-on coding ability in Python.
• A broad knowledge of technical solutions, design patterns, and code for medium/complex applications deployed in a clustered environment.
• Mandatory skills with Docker, Kubernetes and AWS Amazon Web Services.
• Solid SQL experience, authoring complex queries and Data Definition Language (DDL).
• Experience with Linux/Unix.
• Working knowledge of API-based integration for modern systems.
• Experience querying big data, using tools like, Redshift, Hive, Spark, Presto etc.
• Experience with Data Pipelines Orchestration tools like Airflow or Argo Workflows.
• Experience with Source control in Bitbucket or similar.
• Exposure to security concepts and best practices.
• DataIku experience (desirable)
"""


# Loop through all files in the folder
for filename in os.listdir(folder_path):
     if filename.endswith(".pdf") or filename.endswith(".docx"):
          file_path = os.path.join(folder_path, filename)
          resume_text = extractTextPDForDOCX(file_path, filename)
          print("--------")
          # print(resume_text)

          prompt = f"""
          Can you rate how well this Resume in '''{resume_text}''' matches with this Job description in ####{jobdesc}#### \
          You can follow these steps:\
          Step 1 - Extract key skills and match\
          Step 2 - Summarize and match\
          Step 3 - Identify Mandatory experience and mention them\
          Step 4 - Find out his seniority as name as Junior if less than 3 yrs of total experience and Senior if more than 3 yrs of total experience
          Step 5 - Refer the Objective in Job Description and pint it out and its the most impotnat criteria \
          Step 6 - Based on the above steps do an objective ranking for this person in scale of 1 to 100 \
          where 1 is no match (negative), 50 is ok match (neutral) and 100 is excellent match (positive).\
          
          The Resume is delimited with triple ''' \
          The Job Description is delimited with #### \
          
          Just answer with 3 things
          Name, Email id
          - Experience - Junior Data Engineer or Senior Data Engineer
          - Rating : The numeric scale (as above) out of 100 (so you can say x / 100)
          - Suitability : Less than 100 words description to justify the numeric scale and do mention about mandatory skills 
          """
          response = get_completion(prompt)
          print(response)
          print("--------")
